# Unified Energy Forecasting Models

This notebook trains and evaluates MLP, LSTM, and CNN models for Italian energy generation forecasting.

The models are imported from `models.py` and training is handled by `trainer.py`.


## 1. Imports and Setup


In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from math import sqrt

# Setup paths for imports
# Get current working directory (should be FREQ_NETS_TF when running notebook)
current_dir = os.getcwd()

# Determine notebook directory
# If current directory ends with FREQ_NETS_TF, use it; otherwise try to find it
if os.path.basename(current_dir) == 'FREQ_NETS_TF':
    notebook_dir = current_dir
    parent_dir = os.path.dirname(notebook_dir)
else:
    # Try to find FREQ_NETS_TF in the path
    if 'FREQ_NETS_TF' in current_dir:
        parts = current_dir.split('FREQ_NETS_TF')
        notebook_dir = os.path.join(parts[0], 'FREQ_NETS_TF')
        parent_dir = os.path.dirname(notebook_dir)
    else:
        # Assume we need to go up one level or find src
        # This handles cases where notebook is run from project root
        notebook_dir = os.path.join(current_dir, 'src', 'FREQ_NETS_TF') if os.path.exists(os.path.join(current_dir, 'src', 'FREQ_NETS_TF')) else current_dir
        parent_dir = os.path.dirname(notebook_dir) if os.path.basename(notebook_dir) == 'FREQ_NETS_TF' else notebook_dir

# Add paths to sys.path
if notebook_dir not in sys.path:
    sys.path.insert(0, notebook_dir)
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

# Import from our modules (in same directory as notebook)
from models import create_model, ModelBuilder, DEFAULT_CONFIGS
from trainer import ModelTrainer, DataPreparator, run_all_models

# Import preprocessing functions (from parent directory)
from data_preprocessing import preprocess_pipeline, retrieve_data, handle_missing_values, data_and_aggregator, businesshour_and_we_generation

warnings.filterwarnings('ignore')
%matplotlib inline

print("✓ Imports successful!")
print(f"  Current directory: {current_dir}")
print(f"  Notebook directory: {notebook_dir}")
print(f"  Parent directory: {parent_dir}")


## 2. Data Loading and Preprocessing


In [ ]:
# Configuration
DATA_PATH = "data"  # Change this to your data path
DOWNLOAD_FROM_DRIVE = False  # Set to True to download from Google Drive
DRIVE_FOLDER_ID = "1fgXJNVg3MUu8Vx8kAthW4dAih9rKme2H"  # Google Drive folder ID
YEARS = [2016, 2017, 2018, 2019, 2020, 2021]  # Years to load

print("Loading and preprocessing data...")

# Load and preprocess data
if DOWNLOAD_FROM_DRIVE:
    data = preprocess_pipeline(
        data_path=DATA_PATH,
        download_from_drive=True,
        drive_folder_id=DRIVE_FOLDER_ID,
        years=YEARS
    )
else:
    # Use existing local files
    data = retrieve_data(DATA_PATH, years=YEARS)
    data = handle_missing_values(data)
    data = data_and_aggregator(data)
    data = businesshour_and_we_generation(data)
    data = data.dropna()

print(f"Data loaded: {data.shape[0]} samples, {data.shape[1]} features")
print(f"Date range: {data.index.min()} to {data.index.max()}")
print(f"\nColumns: {list(data.columns)}")


## 3. Training Configuration


In [ ]:
# Training parameters
N_TRAIN = 35064  # 3 years of hourly data (24 * 365 * 3)
N_FORECAST_STEPS = 24  # Forecast 24 hours ahead
EPOCHS = 100
BATCH_SIZE = 168
LEARNING_RATE = 0.003
PATIENCE = 3  # Early stopping patience

print(f"Training samples: {N_TRAIN}")
print(f"Test samples: {len(data) - N_TRAIN}")
print(f"Forecast horizon: {N_FORECAST_STEPS} hours")


## 4. Visualization Functions


In [ ]:
def plot_predictions_vs_actual(truth, predictions, title="Predictions vs Actual"):
    """Plot predicted vs actual values."""
    plt.figure(figsize=(15, 6))
    plt.plot(truth, label='Actual', alpha=0.7)
    plt.plot(predictions, label='Predicted', color='red', alpha=0.7)
    plt.title(title)
    plt.xlabel('Time')
    plt.ylabel('Total Aggregated Energy (MW)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()


## 5. MLP Model Training

### 5.1 MLP with Time Series Only


In [ ]:
print("="*60)
print("Training MLP with Time Series Only")
print("="*60)

trainer_mlp_ts = ModelTrainer(
    model_type='mlp',
    feature_type='ts_only',
    window_size=24,  # Use 24 lags
    n_forecast_steps=N_FORECAST_STEPS,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    patience=PATIENCE,
    verbose=True
)

results_mlp_ts = trainer_mlp_ts.train(data, n_train=N_TRAIN, save_model=False)

print(f"\nFinal Results:")
print(f"RMSE: {results_mlp_ts['overall_rmse']:.2f}")
print(f"MAE: {results_mlp_ts['overall_mae']:.2f}")
print(f"R²: {results_mlp_ts['overall_r2']:.4f}")


In [ ]:
# Visualize predictions
plot_predictions_vs_actual(
    results_mlp_ts['truth'],
    results_mlp_ts['predictions'],
    title="MLP - Time Series Only: Predictions vs Actual"
)


### 5.2 MLP with Weekend Dummies


In [ ]:
print("="*60)
print("Training MLP with Weekend Dummies")
print("="*60)

trainer_mlp_we = ModelTrainer(
    model_type='mlp',
    feature_type='weekend',
    window_size=1,  # Use lag_1 with dummies
    n_forecast_steps=N_FORECAST_STEPS,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    patience=PATIENCE,
    verbose=True
)

results_mlp_we = trainer_mlp_we.train(data, n_train=N_TRAIN, save_model=False)

print(f"\nFinal Results:")
print(f"RMSE: {results_mlp_we['overall_rmse']:.2f}")
print(f"MAE: {results_mlp_we['overall_mae']:.2f}")
print(f"R²: {results_mlp_we['overall_r2']:.4f}")


In [ ]:
# Visualize predictions
plot_predictions_vs_actual(
    results_mlp_we['truth'],
    results_mlp_we['predictions'],
    title="MLP - Weekend Dummies: Predictions vs Actual"
)


### 5.3 MLP with Business Hour Dummy


In [ ]:
print("="*60)
print("Training MLP with Business Hour Dummy")
print("="*60)

trainer_mlp_bh = ModelTrainer(
    model_type='mlp',
    feature_type='business_hour',
    window_size=1,  # Use lag_1 with dummy
    n_forecast_steps=N_FORECAST_STEPS,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    patience=PATIENCE,
    verbose=True
)

results_mlp_bh = trainer_mlp_bh.train(data, n_train=N_TRAIN, save_model=False)

print(f"\nFinal Results:")
print(f"RMSE: {results_mlp_bh['overall_rmse']:.2f}")
print(f"MAE: {results_mlp_bh['overall_mae']:.2f}")
print(f"R²: {results_mlp_bh['overall_r2']:.4f}")


In [ ]:
# Visualize predictions
plot_predictions_vs_actual(
    results_mlp_bh['truth'],
    results_mlp_bh['predictions'],
    title="MLP - Business Hour: Predictions vs Actual"
)


## 6. LSTM Model Training

### 6.1 LSTM with Time Series Only


In [ ]:
print("="*60)
print("Training LSTM with Time Series Only")
print("="*60)

trainer_lstm_ts = ModelTrainer(
    model_type='lstm',
    feature_type='ts_only',
    window_size=1,  # Sequence length of 1
    n_forecast_steps=N_FORECAST_STEPS,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    patience=PATIENCE,
    verbose=True
)

results_lstm_ts = trainer_lstm_ts.train(data, n_train=N_TRAIN, save_model=False)

print(f"\nFinal Results:")
print(f"RMSE: {results_lstm_ts['overall_rmse']:.2f}")
print(f"MAE: {results_lstm_ts['overall_mae']:.2f}")
print(f"R²: {results_lstm_ts['overall_r2']:.4f}")


In [ ]:
# Visualize predictions
plot_predictions_vs_actual(
    results_lstm_ts['truth'],
    results_lstm_ts['predictions'],
    title="LSTM - Time Series Only: Predictions vs Actual"
)


### 6.2 LSTM with Weekend Dummies


In [ ]:
print("="*60)
print("Training LSTM with Weekend Dummies")
print("="*60)

trainer_lstm_we = ModelTrainer(
    model_type='lstm',
    feature_type='weekend',
    window_size=1,
    n_forecast_steps=N_FORECAST_STEPS,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    patience=PATIENCE,
    verbose=True
)

results_lstm_we = trainer_lstm_we.train(data, n_train=N_TRAIN, save_model=False)

print(f"\nFinal Results:")
print(f"RMSE: {results_lstm_we['overall_rmse']:.2f}")
print(f"MAE: {results_lstm_we['overall_mae']:.2f}")
print(f"R²: {results_lstm_we['overall_r2']:.4f}")


In [ ]:
# Visualize predictions
plot_predictions_vs_actual(
    results_lstm_we['truth'],
    results_lstm_we['predictions'],
    title="LSTM - Weekend Dummies: Predictions vs Actual"
)


### 6.3 LSTM with Business Hour Dummy


In [ ]:
print("="*60)
print("Training LSTM with Business Hour Dummy")
print("="*60)

trainer_lstm_bh = ModelTrainer(
    model_type='lstm',
    feature_type='business_hour',
    window_size=1,
    n_forecast_steps=N_FORECAST_STEPS,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    patience=PATIENCE,
    verbose=True
)

results_lstm_bh = trainer_lstm_bh.train(data, n_train=N_TRAIN, save_model=False)

print(f"\nFinal Results:")
print(f"RMSE: {results_lstm_bh['overall_rmse']:.2f}")
print(f"MAE: {results_lstm_bh['overall_mae']:.2f}")
print(f"R²: {results_lstm_bh['overall_r2']:.4f}")


In [ ]:
# Visualize predictions
plot_predictions_vs_actual(
    results_lstm_bh['truth'],
    results_lstm_bh['predictions'],
    title="LSTM - Business Hour: Predictions vs Actual"
)


## 7. CNN Model Training

### 7.1 CNN with Time Series Only


In [ ]:
print("="*60)
print("Training CNN with Time Series Only")
print("="*60)

trainer_cnn_ts = ModelTrainer(
    model_type='cnn',
    feature_type='ts_only',
    window_size=2,  # Window of 2 timesteps
    n_forecast_steps=N_FORECAST_STEPS,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    patience=PATIENCE,
    verbose=True
)

results_cnn_ts = trainer_cnn_ts.train(data, n_train=N_TRAIN, save_model=False)

print(f"\nFinal Results:")
print(f"RMSE: {results_cnn_ts['overall_rmse']:.2f}")
print(f"MAE: {results_cnn_ts['overall_mae']:.2f}")
print(f"R²: {results_cnn_ts['overall_r2']:.4f}")


In [ ]:
# Visualize predictions
plot_predictions_vs_actual(
    results_cnn_ts['truth'],
    results_cnn_ts['predictions'],
    title="CNN - Time Series Only: Predictions vs Actual"
)


### 7.2 CNN with Weekend Dummies


In [ ]:
print("="*60)
print("Training CNN with Weekend Dummies")
print("="*60)

trainer_cnn_we = ModelTrainer(
    model_type='cnn',
    feature_type='weekend',
    window_size=2,
    n_forecast_steps=N_FORECAST_STEPS,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    patience=PATIENCE,
    verbose=True
)

results_cnn_we = trainer_cnn_we.train(data, n_train=N_TRAIN, save_model=False)

print(f"\nFinal Results:")
print(f"RMSE: {results_cnn_we['overall_rmse']:.2f}")
print(f"MAE: {results_cnn_we['overall_mae']:.2f}")
print(f"R²: {results_cnn_we['overall_r2']:.4f}")


In [ ]:
# Visualize predictions
plot_predictions_vs_actual(
    results_cnn_we['truth'],
    results_cnn_we['predictions'],
    title="CNN - Weekend Dummies: Predictions vs Actual"
)


### 7.3 CNN with Business Hour Dummy


In [ ]:
print("="*60)
print("Training CNN with Business Hour Dummy")
print("="*60)

trainer_cnn_bh = ModelTrainer(
    model_type='cnn',
    feature_type='business_hour',
    window_size=2,
    n_forecast_steps=N_FORECAST_STEPS,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    patience=PATIENCE,
    verbose=True
)

results_cnn_bh = trainer_cnn_bh.train(data, n_train=N_TRAIN, save_model=False)

print(f"\nFinal Results:")
print(f"RMSE: {results_cnn_bh['overall_rmse']:.2f}")
print(f"MAE: {results_cnn_bh['overall_mae']:.2f}")
print(f"R²: {results_cnn_bh['overall_r2']:.4f}")


In [ ]:
# Visualize predictions
plot_predictions_vs_actual(
    results_cnn_bh['truth'],
    results_cnn_bh['predictions'],
    title="CNN - Business Hour: Predictions vs Actual"
)


## 8. Results Comparison


In [ ]:
# Collect all results
all_results = [
    {'Model': 'MLP', 'Features': 'TS Only', 'RMSE': results_mlp_ts['overall_rmse'], 
     'MAE': results_mlp_ts['overall_mae'], 'R²': results_mlp_ts['overall_r2']},
    {'Model': 'MLP', 'Features': 'Weekend', 'RMSE': results_mlp_we['overall_rmse'], 
     'MAE': results_mlp_we['overall_mae'], 'R²': results_mlp_we['overall_r2']},
    {'Model': 'MLP', 'Features': 'Business Hour', 'RMSE': results_mlp_bh['overall_rmse'], 
     'MAE': results_mlp_bh['overall_mae'], 'R²': results_mlp_bh['overall_r2']},
    {'Model': 'LSTM', 'Features': 'TS Only', 'RMSE': results_lstm_ts['overall_rmse'], 
     'MAE': results_lstm_ts['overall_mae'], 'R²': results_lstm_ts['overall_r2']},
    {'Model': 'LSTM', 'Features': 'Weekend', 'RMSE': results_lstm_we['overall_rmse'], 
     'MAE': results_lstm_we['overall_mae'], 'R²': results_lstm_we['overall_r2']},
    {'Model': 'LSTM', 'Features': 'Business Hour', 'RMSE': results_lstm_bh['overall_rmse'], 
     'MAE': results_lstm_bh['overall_mae'], 'R²': results_lstm_bh['overall_r2']},
    {'Model': 'CNN', 'Features': 'TS Only', 'RMSE': results_cnn_ts['overall_rmse'], 
     'MAE': results_cnn_ts['overall_mae'], 'R²': results_cnn_ts['overall_r2']},
    {'Model': 'CNN', 'Features': 'Weekend', 'RMSE': results_cnn_we['overall_rmse'], 
     'MAE': results_cnn_we['overall_mae'], 'R²': results_cnn_we['overall_r2']},
    {'Model': 'CNN', 'Features': 'Business Hour', 'RMSE': results_cnn_bh['overall_rmse'], 
     'MAE': results_cnn_bh['overall_mae'], 'R²': results_cnn_bh['overall_r2']},
]

results_df = pd.DataFrame(all_results)
results_df = results_df.sort_values('RMSE')

print("="*60)
print("ALL MODELS COMPARISON (Sorted by RMSE)")
print("="*60)
print(results_df.to_string(index=False))


In [ ]:
# Visualize comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# RMSE comparison
results_pivot_rmse = results_df.pivot(index='Model', columns='Features', values='RMSE')
results_pivot_rmse.plot(kind='bar', ax=axes[0], color=['skyblue', 'lightcoral', 'lightgreen'])
axes[0].set_title('RMSE Comparison by Model')
axes[0].set_ylabel('RMSE')
axes[0].tick_params(axis='x', rotation=45)
axes[0].legend(title='Features')
axes[0].grid(True, alpha=0.3)

# MAE comparison
results_pivot_mae = results_df.pivot(index='Model', columns='Features', values='MAE')
results_pivot_mae.plot(kind='bar', ax=axes[1], color=['skyblue', 'lightcoral', 'lightgreen'])
axes[1].set_title('MAE Comparison by Model')
axes[1].set_ylabel('MAE')
axes[1].tick_params(axis='x', rotation=45)
axes[1].legend(title='Features')
axes[1].grid(True, alpha=0.3)

# R² comparison
results_pivot_r2 = results_df.pivot(index='Model', columns='Features', values='R²')
results_pivot_r2.plot(kind='bar', ax=axes[2], color=['skyblue', 'lightcoral', 'lightgreen'])
axes[2].set_title('R² Comparison by Model')
axes[2].set_ylabel('R²')
axes[2].tick_params(axis='x', rotation=45)
axes[2].legend(title='Features')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Best model
best_model = results_df.iloc[0]
print("="*60)
print("BEST MODEL")
print("="*60)
print(f"Model: {best_model['Model']}")
print(f"Features: {best_model['Features']}")
print(f"RMSE: {best_model['RMSE']:.2f}")
print(f"MAE: {best_model['MAE']:.2f}")
print(f"R²: {best_model['R²']:.4f}")


## 9. Optional: Run All Models Automatically

Alternatively, you can use the `run_all_models()` function to automatically run all configurations:


In [ ]:
# Alternative: Use the run_all_models function for automatic execution
# Uncomment to use:

# results_df_auto = run_all_models(
#     data_path=DATA_PATH,
#     download_from_drive=DOWNLOAD_FROM_DRIVE,
#     drive_folder_id=DRIVE_FOLDER_ID,
#     years=YEARS,
#     n_train=N_TRAIN,
#     save_results=True,
#     results_path="results"
# )
# 
# print(results_df_auto)
